In [1]:
# app_rag_cohere.py
import os
from dotenv import load_dotenv

# LangChain imports (latest packages)
from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# Cohere integration
from langchain_cohere import ChatCohere, CohereEmbeddings

# Pinecone vector store
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings
from pinecone import Pinecone 

c:\Users\Hp\anaconda3\envs\mchatbot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
index_name = "medical-chatbot"
 

if not COHERE_API_KEY:
    raise RuntimeError("Set COHERE_API_KEY in .env")

# 1) load & split PDF files
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(r"C:\Users\Hp\OneDrive\Desktop\Medical-Chatbot-by-using-Vector-Database-Concept-s\data", glob="*.pdf", loader_cls=PyPDFLoader)
docs = loader.load()

docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(docs)

In [3]:
print("length of my chunk:", len(chunks))

length of my chunk: 2657


In [4]:
cohere_emb = CohereEmbeddings(model="embed-english-light-v3.0", cohere_api_key=COHERE_API_KEY)

In [5]:
pc = Pinecone(api_key=PINECONE_API_KEY)
#index_name = "medical-chatbot"
index = pc.Index(index_name)

vec_store = PineconeVectorStore(index=index, embedding=cohere_emb)

# Batch insert documents
batch_size = 50
for i in range(0, len(chunks), batch_size):
    batch = chunks[i:i+batch_size]
    vec_store.add_documents(batch)


Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised TooManyRequestsError: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin', 'x-accel-expires': '0', 'x-debug-trace-id': '83f27107b23d1b85db1dd2dad3165501', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '100', 'x-trial-endpoint-call-remaining': '88', 'date': 'Tue, 02 Sep 2025 16:52:57 GMT', 'content-length': '124', 'x-envoy-upstream-service-time': '55', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'}, status_code: 429, body: {'id': 'd5daaccc-e535-43b6-96b1-5c3a27deaba5', 'message': 'trial token rate limit exceeded, limit is 100000 tokens per minute'}.


In [6]:
qa_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are a medical assistant. 
Use ONLY the following context to answer the user's question. 
If the answer cannot be found in the context, reply with:
"I could not find that information in the provided document."

Context:
{context}

Question:
{question}

Answer:
""",
)

# 4) Retriever
retriever = vec_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

# 5) Cohere LLM
chat = ChatCohere(
    cohere_api_key=COHERE_API_KEY,
    model="command-a-03-2025"
)

# 6) Retrieval QA with custom prompt
qa = RetrievalQA.from_chain_type(
    llm=chat,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": qa_prompt}
)

# 7) Interactive loop
print("📘 Medical PDF Chatbot is ready! Type 'exit' to quit.\n")
while True:
    try:
        query = input("❓ Your question: ")
        if query.lower() in ["exit", "quit", "q"]:
            print("👋 Goodbye!")
            break
        answer = qa.run(query)
        print(f"💡 Answer: {answer}\n")
    except KeyboardInterrupt:
        print("\n👋 Goodbye!")
        break

📘 Medical PDF Chatbot is ready! Type 'exit' to quit.



C:\Users\Hp\AppData\Local\Temp\ipykernel_15616\1697103079.py:47: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa.run(query)


💡 Answer: Diabetes mellitus (DM) is the most common disease of the endocrine system, characterized by abnormally high blood glucose levels. It results from either an absolute deficiency of insulin (type 1 diabetes) or from subnormal target cell response to insulin (insulin resistance) combined with a failure to compensate for this insulin resistance by producing higher concentrations of insulin (relative insulin deficiency) (type 2 diabetes).

👋 Goodbye!
